# Criptografia com diferentes modos do AES: 

Neste notebook, vamos explorar como diferentes modos de operação do algoritmo AES (Advanced Encryption Standard) afetam a criptografia de imagens.

Veremos na prática por que alguns modos são inadequados para certos tipos de dados e como escolher o modo correto para cada situação.

- **AES-ECB**
- **AES-CBC**
- **AES-CTR**
- **AES-GCM**


## Bibliotecas

In [ ]:
!pip install cryptography pillow requests numpy matplotlib

In [ ]:
import requests
from PIL import Image
import numpy as np
import matplotlib.pyplot as plt
from cryptography.hazmat.primitives.ciphers import Cipher, algorithms, modes
from cryptography.hazmat.primitives.ciphers.aead import AESGCM
from cryptography.hazmat.backends import default_backend
import os
import io
from typing import Tuple, Optional
import hashlib
import time

## Metodos

### Carregamento e Preparação da Imagem

In [ ]:
def load_image_from_url(url: str) -> np.ndarray:

    try:
        response = requests.get(url)
        response.raise_for_status()
        
        image = Image.open(io.BytesIO(response.content))
        image = image.convert('RGB')
        
        if max(image.size) > 512:
            image.thumbnail((512, 512), Image.Resampling.LANCZOS)
        
        return np.array(image)
    
    except Exception as e:
        print(f"Erro ao carregar imagem: {e}")
        return create_test_pattern()

# Criar padrões que mostram bem os efeitos da criptografia (caso nao tenha url)
def create_test_pattern() -> np.ndarray:
    img = np.zeros((256, 256, 3), dtype=np.uint8)
    
    img[50:100, 50:100] = [255, 0, 0]  
    img[150:200, 150:200] = [0, 255, 0]
    img[100:150, 50:200] = [0, 0, 255]
    
    for i in range(0, 256, 20):
        img[i:i+10, :] = [255, 255, 255]
    
    return img


### Funções Auxiliares para Criptografia

- Preparar dados para criptografia (padding)
- Gerar chaves e IVs seguros
- Medir performance das operações

In [ ]:
def pad_data(data: bytes, block_size: int = 16) -> bytes:
    """
    Aplica padding PKCS7 aos dados.
    
    O PKCS7 é essencial para modos como CBC que operam em blocos fixos.
    Exemplo: se faltam 3 bytes para completar um bloco, adiciona 3 bytes com valor 3.
    """
    padding_length = block_size - (len(data) % block_size)
    padding = bytes([padding_length] * padding_length)
    return data + padding

def unpad_data(data: bytes) -> bytes:
    """
    Remove padding PKCS7 dos dados descriptografados.
    """
    if not data:
        return data
    padding_length = data[-1]
    return data[:-padding_length]

def generate_key() -> bytes:
    """
    Gera uma chave AES-256 criptograficamente segura.
    """
    return os.urandom(32)  # 256 bits

def generate_iv() -> bytes:
    """
    Gera um IV (Initialization Vector) aleatório.
    """
    return os.urandom(16)  # 128 bits

def image_to_bytes(image: np.ndarray) -> bytes:
    """
    Converte imagem numpy para bytes.
    """
    return image.tobytes()

def bytes_to_image(data: bytes, shape: Tuple[int, int, int]) -> np.ndarray:
    """
    Converte bytes de volta para imagem numpy.
    """
    return np.frombuffer(data, dtype=np.uint8).reshape(shape)

def measure_time(func):
    """
    Decorator para medir tempo de execução.
    """
    def wrapper(*args, **kwargs):
        start_time = time.time()
        result = func(*args, **kwargs)
        end_time = time.time()
        print(f"{func.__name__} executado em {end_time - start_time:.4f} segundos")
        return result
    return wrapper

### Variaveis

In [ ]:
image_url = 'https://static.wikia.nocookie.net/turma-do-kamil/images/5/53/A5f7322cdf21c4de36c7e2c48c926e4c433fe5e4_hq.jpg/revision/latest?cb=20240814003202&path-prefix=pt-br'

In [ ]:
if not image_url.strip():
    print("Usando padrão de teste...")
    original_image = create_test_pattern()
else:
    print(f"Carregando imagem de: {image_url}")
    original_image = load_image_from_url(image_url)

print(f"Imagem carregada: {original_image.shape}")

# Mostrar imagem original
plt.figure(figsize=(8, 6))
plt.imshow(original_image)
plt.title("Imagem Original")
plt.axis('off')
plt.show()

In [ ]:
master_key = generate_key()
print(f"Chave : {master_key.hex()}")

## Cenários

### 🚨 AES-ECB

🏥 **Sistema Hospitalar** :

Imagine um hospital que usa ECB para criptografar imagens de raios-X. Mesmo criptografadas, um atacante poderia:
- Identificar padrões anatômicos
- Reconhecer tipos de fraturas
- Comparar exames diferentes do mesmo paciente

#### Metodos

In [ ]:
@measure_time
def encrypt_ecb(data: bytes, key: bytes) -> bytes:
    # ECB requer padding pois opera em blocos fixos
    padded_data = pad_data(data)
    
    cipher = Cipher(
        algorithms.AES(key),
        modes.ECB(),
        backend=default_backend()
    )
    
    encryptor = cipher.encryptor()
    ciphertext = encryptor.update(padded_data) + encryptor.finalize()
    
    return ciphertext

@measure_time
def decrypt_ecb(ciphertext: bytes, key: bytes) -> bytes:
    cipher = Cipher(
        algorithms.AES(key),
        modes.ECB(),
        backend=default_backend()
    )
    
    decryptor = cipher.decryptor()
    padded_plaintext = decryptor.update(ciphertext) + decryptor.finalize()
    
    return unpad_data(padded_plaintext)

#### Exemplo

In [ ]:
# Converter imagem para bytes
image_bytes = image_to_bytes(original_image)
print(f"Tamanho da imagem em bytes: {len(image_bytes)}")

# Criptografar com ECB
ecb_encrypted = encrypt_ecb(image_bytes, master_key)

In [ ]:
image_bytes[:64]

In [ ]:
ecb_encrypted[:64]

🔓 Descriptografando com AES-ECB...

In [ ]:
ecb_decrypted = decrypt_ecb(ecb_encrypted, master_key)
ecb_decrypted[:64]

Reconstruir imagem criptografada

In [ ]:
try:
    ecb_encrypted_truncated = ecb_encrypted[:len(image_bytes)]
    ecb_image_encrypted = bytes_to_image(ecb_encrypted_truncated, original_image.shape)
    
    ecb_image_decrypted = bytes_to_image(ecb_decrypted, original_image.shape)
    
    # Verificar se a descriptografia foi bem-sucedida
    if np.array_equal(original_image, ecb_image_decrypted):
        pass
    else:
        print("Erro na descriptografia ECB")
    
except Exception as e:
    print(f"Erro na reconstrução: {e}")
    ecb_image_encrypted = original_image  # Fallback
    ecb_image_decrypted = original_image

# Visualizar resultados
fig, axes = plt.subplots(1, 3, figsize=(15, 5))

axes[0].imshow(original_image)
axes[0].set_title("Original")
axes[0].axis('off')

axes[1].imshow(ecb_image_encrypted)
axes[1].set_title("AES-ECB Criptografado\n(Padrões ainda visíveis!)")
axes[1].axis('off')

axes[2].imshow(ecb_image_decrypted)
axes[2].set_title("AES-ECB Descriptografado")
axes[2].axis('off')

plt.tight_layout()
plt.show()

⚠️ ECB é Problemático:
1. **Padrões visíveis**: Estruturas repetitivas permanecem visíveis
2. **Análise estatística**: Blocos frequentes podem ser identificados
3. **Ataques de replay**: Blocos podem ser reutilizados maliciosamente

### AES-CBC: Adicionando Randomização

🏢 Caso Prático - Backup Corporativo:
Uma empresa precisa fazer backup de documentos confidenciais:
- **Confidencialidade**: CBC esconde completamente os padrões
- **Determinismo**: O mesmo documento sempre gera criptogramas diferentes (devido ao IV aleatório)
- **Integridade**: Alterações em qualquer bloco afetam todos os blocos seguintes

#### Metodos

In [ ]:
@measure_time
def encrypt_cbc(data: bytes, key: bytes, iv: Optional[bytes] = None) -> Tuple[bytes, bytes]:
    if iv is None:
        iv = generate_iv()
    
    # CBC requer padding
    padded_data = pad_data(data)
    
    cipher = Cipher(
        algorithms.AES(key),
        modes.CBC(iv),
        backend=default_backend()
    )
    
    encryptor = cipher.encryptor()
    ciphertext = encryptor.update(padded_data) + encryptor.finalize()
    
    return ciphertext, iv

@measure_time
def decrypt_cbc(ciphertext: bytes, key: bytes, iv: bytes) -> bytes:
    cipher = Cipher(
        algorithms.AES(key),
        modes.CBC(iv),
        backend=default_backend()
    )
    
    decryptor = cipher.decryptor()
    padded_plaintext = decryptor.update(ciphertext) + decryptor.finalize()
    
    return unpad_data(padded_plaintext)

#### Exemplo

In [ ]:
# Criptografar a mesma imagem duas vezes
cbc_encrypted1, iv1 = encrypt_cbc(image_bytes, master_key)
cbc_encrypted2, iv2 = encrypt_cbc(image_bytes, master_key)

In [ ]:
print(f"IV 1: {iv1.hex()[:16]}...")
print(f"IV 2: {iv2.hex()[:16]}...")
print(f"iv1 == iv2? {cbc_encrypted1 == cbc_encrypted2}")

In [ ]:
# Descriptografar ambos
cbc_decrypted1 = decrypt_cbc(cbc_encrypted1, master_key, iv1)
cbc_decrypted2 = decrypt_cbc(cbc_encrypted2, master_key, iv2)


In [ ]:
# Verificar se ambas descriptografias estão corretas
print(f"Descriptografia 1 OK? {cbc_decrypted1 == image_bytes}")
print(f"Descriptografia 2 OK? {cbc_decrypted2 == image_bytes}")

In [ ]:
# Visualizar uma das criptografias
try:
    cbc_encrypted_truncated = cbc_encrypted1[:len(image_bytes)]
    cbc_image_encrypted = bytes_to_image(cbc_encrypted_truncated, original_image.shape)
    cbc_image_decrypted = bytes_to_image(cbc_decrypted1, original_image.shape)
    
except Exception as e:
    print(f"Erro na reconstrução: {e}")
    cbc_image_encrypted = np.random.randint(0, 256, original_image.shape, dtype=np.uint8)
    cbc_image_decrypted = original_image

# Comparar ECB vs CBC
fig, axes = plt.subplots(2, 3, figsize=(15, 10))

# Linha 1: ECB
axes[0,0].imshow(original_image)
axes[0,0].set_title("Original")
axes[0,0].axis('off')

axes[0,1].imshow(ecb_image_encrypted)
axes[0,1].set_title("AES-ECB\n(Padrões visíveis)")
axes[0,1].axis('off')

axes[0,2].imshow(ecb_image_decrypted)
axes[0,2].set_title("ECB Descriptografado")
axes[0,2].axis('off')

# Linha 2: CBC
axes[1,0].imshow(original_image)
axes[1,0].set_title("Original")
axes[1,0].axis('off')

axes[1,1].imshow(cbc_image_encrypted)
axes[1,1].set_title("AES-CBC\n(Completamente aleatório)")
axes[1,1].axis('off')

axes[1,2].imshow(cbc_image_decrypted)
axes[1,2].set_title("CBC Descriptografado")
axes[1,2].axis('off')

plt.tight_layout()
plt.show()


Vantagens do CBC:
1. **Sem padrões visíveis**: Dados idênticos produzem criptogramas diferentes
2. **Propagação de erros**: Alterações se propagam, ajudando na detecção
3. **Padrão da indústria**: Amplamente testado e confiável

### AES-CTR: Modo de Fluxo

📺 Caso Prático - Streaming de Vídeo:
Uma plataforma de streaming precisa criptografar vídeos em tempo real:
- **Paralelização**: Múltiplos blocos podem ser processados simultaneamente
- **Acesso aleatório**: Pode pular para qualquer parte do vídeo sem descriptografar tudo
- **Performance**: Mais rápido que CBC para grandes volumes de dados
- **Sem padding**: Trabalha com qualquer tamanho de dados

In [ ]:
@measure_time
def encrypt_ctr(data: bytes, key: bytes, nonce: Optional[bytes] = None) -> Tuple[bytes, bytes]:
    if nonce is None:
        nonce = os.urandom(16)  # 128 bits
    
    # CTR não precisa de padding!
    cipher = Cipher(
        algorithms.AES(key),
        modes.CTR(nonce),
        backend=default_backend()
    )
    
    encryptor = cipher.encryptor()
    ciphertext = encryptor.update(data) + encryptor.finalize()
    
    return ciphertext, nonce

@measure_time
def decrypt_ctr(ciphertext: bytes, key: bytes, nonce: bytes) -> bytes:
    cipher = Cipher(
        algorithms.AES(key),
        modes.CTR(nonce),
        backend=default_backend()
    )
    
    decryptor = cipher.decryptor()
    plaintext = decryptor.update(ciphertext) + decryptor.finalize()
    
    return plaintext

def demonstrate_ctr_random_access(data: bytes, key: bytes, nonce: bytes):
    
    # Criptografar dados completos
    full_cipher = Cipher(algorithms.AES(key), modes.CTR(nonce), backend=default_backend())
    full_encryptor = full_cipher.encryptor()
    full_encrypted = full_encryptor.update(data) + full_encryptor.finalize()
    
    # Simular acesso a uma parte específica (por exemplo, do byte 1000 ao 2000)
    start_pos = min(1000, len(data) // 2)
    end_pos = min(2000, len(data))
    
    if start_pos < end_pos:
        partial_encrypted = full_encrypted[start_pos:end_pos]
        
        print(f"Acessando bytes {start_pos} a {end_pos} diretamente")
        print(f"Tamanho da parte acessada: {len(partial_encrypted)} bytes")


#### Exemplo

In [ ]:
# Criptografar com CTR
ctr_encrypted, nonce = encrypt_ctr(image_bytes, master_key)

In [ ]:
ctr_decrypted = decrypt_ctr(ctr_encrypted, master_key, nonce)

In [ ]:
# Verificar integridade
print(f"CTR Descriptografia? {ctr_decrypted == image_bytes}")
print(f"Tamanho original: {len(image_bytes)} bytes")
print(f"Tamanho criptografado: {len(ctr_encrypted)} bytes (sem padding!)")

In [ ]:
# Demonstrar acesso aleatório
demonstrate_ctr_random_access(image_bytes, master_key, nonce)

In [ ]:
# Visualizar
try:
    ctr_image_encrypted = bytes_to_image(ctr_encrypted, original_image.shape)
    ctr_image_decrypted = bytes_to_image(ctr_decrypted, original_image.shape)
except Exception as e:
    print(f"Erro na reconstrução: {e}")
    ctr_image_encrypted = np.random.randint(0, 256, original_image.shape, dtype=np.uint8)
    ctr_image_decrypted = original_image

# Comparar todos os modos até agora
fig, axes = plt.subplots(2, 4, figsize=(16, 8))

# Linha 1: Originais
axes[0,0].imshow(original_image)
axes[0,0].set_title("Original")
axes[0,0].axis('off')

axes[0,1].imshow(ecb_image_encrypted)
axes[0,1].set_title("ECB\n(Padrões visíveis)")
axes[0,1].axis('off')

axes[0,2].imshow(cbc_image_encrypted)
axes[0,2].set_title("CBC\n(Aleatório)")
axes[0,2].axis('off')

axes[0,3].imshow(ctr_image_encrypted)
axes[0,3].set_title("CTR\n(Aleatório + Eficiente)")
axes[0,3].axis('off')

# Linha 2: Descriptografados
axes[1,0].imshow(original_image)
axes[1,0].set_title("Original")
axes[1,0].axis('off')

axes[1,1].imshow(ecb_image_decrypted)
axes[1,1].set_title("ECB Descriptografado")
axes[1,1].axis('off')

axes[1,2].imshow(cbc_image_decrypted)
axes[1,2].set_title("CBC Descriptografado")
axes[1,2].axis('off')

axes[1,3].imshow(ctr_image_decrypted)
axes[1,3].set_title("CTR Descriptografado")
axes[1,3].axis('off')

plt.tight_layout()
plt.show()

Vantagens do CTR:
1. **Paralelizável**: Blocos independentes permitem processamento paralelo
2. **Sem padding**: Funciona com qualquer tamanho de dados
3. **Acesso aleatório**: Pode descriptografar qualquer parte independentemente
4. **Desempenho**: Melhor em hardware moderno

### AES-GCM: Criptografia Autenticada



🏥 Caso Prático - Prontuários Médicos:

Um hospital precisa armazenar imagens médicas na nuvem:
- **Confidencialidade**: Pacientes não podem ter dados vazados
- **Integridade**: Alterações em exames podem ser fatais
- **Autenticidade**: Precisa garantir que os dados vieram do hospital
- **Compliance**: LGPD/HIPAA exigem controles rigorosos

#### Metodos

In [ ]:
@measure_time
def encrypt_gcm(data: bytes, key: bytes, associated_data: Optional[bytes] = None) -> Tuple[bytes, bytes, bytes]:
    # GCM usa nonce de 96 bits (12 bytes) por padrão
    nonce = os.urandom(12)
    
    aesgcm = AESGCM(key)
    
    # O GCM retorna ciphertext + authentication tag combinados
    ciphertext_with_tag = aesgcm.encrypt(nonce, data, associated_data)
    
    # Separar ciphertext do tag (últimos 16 bytes)
    ciphertext = ciphertext_with_tag[:-16]
    tag = ciphertext_with_tag[-16:]
    
    return ciphertext, nonce, tag

@measure_time
def decrypt_gcm(ciphertext: bytes, key: bytes, nonce: bytes, tag: bytes, associated_data: Optional[bytes] = None) -> bytes:
    aesgcm = AESGCM(key)
    
    # Recombinar ciphertext + tag
    ciphertext_with_tag = ciphertext + tag
    
    # Descriptografar e verificar autenticidade
    plaintext = aesgcm.decrypt(nonce, ciphertext_with_tag, associated_data)
    
    return plaintext

#### Exemplo

Criptografando com AES-GCM (com metadados associados)...

In [ ]:
metadata = f"Paciente: João Silva, Data: 2024-01-15, Tipo: Raio-X, Tamanho: {len(image_bytes)}".encode()

print(f"Metadados: {metadata.decode()}")

In [ ]:
gcm_encrypted, gcm_nonce, gcm_tag = encrypt_gcm(image_bytes, master_key, metadata)

In [ ]:
len(gcm_encrypted), gcm_encrypted[:32]

In [ ]:
print(f"Nonce: {gcm_nonce.hex()}")
print(f"Tag de autenticação: {gcm_tag.hex()}")

 Descriptografando e verificando integridade...

In [ ]:
try:
    gcm_decrypted = decrypt_gcm(gcm_encrypted, master_key, gcm_nonce, gcm_tag, metadata)
    print(f" Arquivo Integro?: {gcm_decrypted == image_bytes}")
except Exception as e:
    print(f"Falha na verificação GCM: {e}")
    gcm_decrypted = image_bytes  # Fallback para visualização

# Demonstrar detecção de integridade
# demonstrate_gcm_integrity()

**Simular alteração maliciosa nos dados**

In [ ]:
# Alterar primeiro byte
encrypted_modificado = bytearray(gcm_encrypted)
encrypted_modificado[0] ^= 0xFF  

In [ ]:
try:
    decrypted = decrypt_gcm(bytes(encrypted_modificado), master_key, nonce, gcm_tag)
    print(f"FALHA DE SEGURANÇA: Alteração não detectada!")
except Exception as e:
    print(f"Alteração detectada com sucesso: {type(e).__name__}")

**Simular alteração no tag de autenticação**

In [ ]:
tag_modificada = bytearray(gcm_tag)
tag_modificada[0] ^= 0xFF

try:
    decrypted = decrypt_gcm(gcm_encrypted, master_key, nonce, bytes(tag_modificada))
    print(f"FALHA DE SEGURANÇA: Alteração do tag não detectada!")
except Exception as e:
    print(f"Alteração do tag detectada: {type(e).__name__}")

In [ ]:
# Visualizar
try:
    gcm_image_encrypted = bytes_to_image(gcm_encrypted, original_image.shape)
    gcm_image_decrypted = bytes_to_image(gcm_decrypted, original_image.shape)
except Exception as e:
    print(f"Erro na reconstrução: {e}")
    gcm_image_encrypted = np.random.randint(0, 256, original_image.shape, dtype=np.uint8)
    gcm_image_decrypted = original_image

# Visualização final: comparação de todos os modos
fig, axes = plt.subplots(3, 5, figsize=(20, 12))

modes = ['Original', 'ECB', 'CBC', 'CTR', 'GCM']
encrypted_images = [original_image, ecb_image_encrypted, cbc_image_encrypted, ctr_image_encrypted, gcm_image_encrypted]
decrypted_images = [original_image, ecb_image_decrypted, cbc_image_decrypted, ctr_image_decrypted, gcm_image_decrypted]

# Linha 1: Imagem original repetida
for i, mode in enumerate(modes):
    axes[0,i].imshow(original_image)
    axes[0,i].set_title(f"Original\n({mode})")
    axes[0,i].axis('off')

# Linha 2: Imagens criptografadas
for i, (mode, img) in enumerate(zip(modes, encrypted_images)):
    axes[1,i].imshow(img)
    if mode == 'Original':
        axes[1,i].set_title("Original")
    elif mode == 'ECB':
        axes[1,i].set_title(f"{mode}\n(Inseguro)")
    else:
        axes[1,i].set_title(f"{mode}\n(Seguro)")
    axes[1,i].axis('off')

# Linha 3: Imagens descriptografadas
for i, (mode, img) in enumerate(zip(modes, decrypted_images)):
    axes[2,i].imshow(img)
    if mode == 'GCM':
        axes[2,i].set_title(f"{mode}\n+ Integridade")
    else:
        axes[2,i].set_title(f"{mode}\nDescriptografado")
    axes[2,i].axis('off')

plt.tight_layout()
plt.show()

##  Análise Comparativa dos modos

Vamos fazer uma análise quantitativa dos diferentes modos de operação